In [6]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy.stats import expon, norm
import numpy as np


In [1]:
# State durations (in days)
state_durations = {
    'Symptoms': lambda: np.random.exponential(scale=2),     # Average 2 days before seeing a doctor
    'PrimaryCare': lambda: np.random.normal(loc=1, scale=0.5),  # Average 1 day for consultation
    'WatchWait': lambda: np.random.uniform(30, 180),        # Between 1 to 6 months
    'ChestXRay': lambda: np.random.normal(loc=7, scale=2),  # Average 7 days to get imaging
    'NoduleDetected': lambda: np.random.normal(loc=1, scale=0.5), # Immediate progression
    'Biopsy': lambda: np.random.normal(loc=14, scale=3),    # Average 2 weeks for biopsy
    'Diagnosis': lambda: 0,                                 # Terminal state, no additional time
    'LostFollowUp': lambda: 0                               # Terminal state, no additional time
}


In [3]:
# States
states = ['Symptoms', 'PrimaryCare', 'WatchWait', 'ChestXRay', 'NoduleDetected', 'Biopsy', 'Diagnosis', 'LostFollowUp']
state_indices = {state: idx for idx, state in enumerate(states)}
num_states = len(states)
print("State Indices:", state_indices)


State Indices: {'Symptoms': 0, 'PrimaryCare': 1, 'WatchWait': 2, 'ChestXRay': 3, 'NoduleDetected': 4, 'Biopsy': 5, 'Diagnosis': 6, 'LostFollowUp': 7}


In [4]:
def simulate_patient_journey_with_time(A, B, start_state, max_steps=20):
    current_state = start_state
    states_visited = [current_state]
    observations_made = []
    time_spent = []
    total_time = 0
    
    for _ in range(max_steps):
        # Generate observation
        obs_probs = B[current_state]
        observation = np.random.choice(range(num_states), p=obs_probs)
        observations_made.append(observation)
        
        # Add time spent in current state
        duration = state_durations[states[current_state]]()
        duration = max(0, duration)  # Ensure non-negative time
        time_spent.append(duration)
        total_time += duration
        
        # Check if current state is absorbing
        if A[current_state, current_state] == 1.0:
            break  # Absorbing state reached
        
        # Transition to next state
        trans_probs = A[current_state]
        next_state = np.random.choice(range(num_states), p=trans_probs)
        states_visited.append(next_state)
        current_state = next_state
        
    return states_visited, observations_made, time_spent, total_time


In [7]:
# Initialize transition matrix with zeros
A = np.zeros((num_states, num_states))

# Define transitions
# From Symptoms
A[state_indices['Symptoms'], state_indices['PrimaryCare']] = 0.8  # 80% see a doctor
A[state_indices['Symptoms'], state_indices['LostFollowUp']] = 0.2  # 20% do not seek care and are lost

# From PrimaryCare
A[state_indices['PrimaryCare'], state_indices['WatchWait']] = 0.6  # 60% cases are watchful waiting
A[state_indices['PrimaryCare'], state_indices['ChestXRay']] = 0.4  # 40% proceed to imaging

# From WatchWait
A[state_indices['WatchWait'], state_indices['ChestXRay']] = 0.3   # 30% eventually get imaging
A[state_indices['WatchWait'], state_indices['LostFollowUp']] = 0.5  # 50% are lost to follow-up
A[state_indices['WatchWait'], state_indices['WatchWait']] = 0.2    # 20% remain in watchful waiting

# From ChestXRay
A[state_indices['ChestXRay'], state_indices['NoduleDetected']] = 0.7  # 70% detect nodules
A[state_indices['ChestXRay'], state_indices['WatchWait']] = 0.2      # 20% return to watchful waiting
A[state_indices['ChestXRay'], state_indices['LostFollowUp']] = 0.1   # 10% are lost to follow-up

# From NoduleDetected
A[state_indices['NoduleDetected'], state_indices['Biopsy']] = 0.9  # 90% proceed to biopsy
A[state_indices['NoduleDetected'], state_indices['LostFollowUp']] = 0.1  # 10% are lost to follow-up

# From Biopsy
A[state_indices['Biopsy'], state_indices['Diagnosis']] = 0.8  # 80% are diagnosed with cancer
A[state_indices['Biopsy'], state_indices['LostFollowUp']] = 0.2  # 20% are lost to follow-up

# Absorbing states
A[state_indices['Diagnosis'], state_indices['Diagnosis']] = 1.0
A[state_indices['LostFollowUp'], state_indices['LostFollowUp']] = 1.0


In [8]:
# Observations
observations = ['Cough', 'PhysicianEncounter', 'NoAction', 'ImagingPerformed', 'NoduleFound', 'BiopsyPerformed', 'CancerDiagnosed', 'NoObservation']
observation_indices = {obs: idx for idx, obs in enumerate(observations)}
num_observations = len(observations)
print("Observation Indices:", observation_indices)


Observation Indices: {'Cough': 0, 'PhysicianEncounter': 1, 'NoAction': 2, 'ImagingPerformed': 3, 'NoduleFound': 4, 'BiopsyPerformed': 5, 'CancerDiagnosed': 6, 'NoObservation': 7}


In [10]:
# Initialize emission probability matrix with zeros
B = np.zeros((num_states, num_observations))

# Define emissions
B[state_indices['Symptoms'], observation_indices['Cough']] = 1.0
B[state_indices['PrimaryCare'], observation_indices['PhysicianEncounter']] = 1.0
B[state_indices['WatchWait'], observation_indices['NoAction']] = 1.0
B[state_indices['ChestXRay'], observation_indices['ImagingPerformed']] = 1.0
B[state_indices['NoduleDetected'], observation_indices['NoduleFound']] = 1.0
B[state_indices['Biopsy'], observation_indices['BiopsyPerformed']] = 1.0
B[state_indices['Diagnosis'], observation_indices['CancerDiagnosed']] = 1.0
B[state_indices['LostFollowUp'], observation_indices['NoObservation']] = 1.0


In [12]:
def simulate_patient_journey_with_time(A, B, start_state, max_steps=20):
    current_state = start_state
    states_visited = [current_state]
    observations_made = []
    time_spent = []
    total_time = 0
    
    for _ in range(max_steps):
        # Generate observation
        obs_probs = B[current_state]
        observation = np.random.choice(range(num_observations), p=obs_probs)
        observations_made.append(observation)
        
        # Add time spent in current state
        duration = state_durations[states[current_state]]()
        duration = max(0, duration)  # Ensure non-negative time
        time_spent.append(duration)
        total_time += duration
        
        # Check if current state is absorbing
        if A[current_state, current_state] == 1.0:
            break  # Absorbing state reached
        
        # Transition to next state
        trans_probs = A[current_state]
        next_state = np.random.choice(range(num_states), p=trans_probs)
        states_visited.append(next_state)
        current_state = next_state
        
    return states_visited, observations_made, time_spent, total_time


In [22]:
# Simulate a patient journey starting from 'Symptoms'
start_state = state_indices['Symptoms']
states_visited, observations_made, time_spent, total_time = simulate_patient_journey_with_time(A, B, start_state)

# Convert indices back to names
visited_states_names = [states[idx] for idx in states_visited]
observations_names = [observations[idx] for idx in observations_made]

print("States Visited:")
for idx, state in enumerate(visited_states_names):
    print(f" - {state} (Time spent: {time_spent[idx]:.2f} days)")

print(f"\nTotal Time to Absorbing State: {total_time:.2f} days")

print("\nObservations Made:")
for obs in observations_names:
    print(f" - {obs}")


States Visited:
 - Symptoms (Time spent: 0.22 days)
 - PrimaryCare (Time spent: 1.21 days)
 - ChestXRay (Time spent: 9.19 days)
 - NoduleDetected (Time spent: 1.90 days)
 - Biopsy (Time spent: 19.26 days)
 - Diagnosis (Time spent: 0.00 days)

Total Time to Absorbing State: 31.78 days

Observations Made:
 - Cough
 - PhysicianEncounter
 - ImagingPerformed
 - NoduleFound
 - BiopsyPerformed
 - CancerDiagnosed


In [23]:
time_spent

[0.21785643847271968,
 1.2101598041232482,
 9.194486003619012,
 1.898793509634749,
 19.260048576950236,
 0]